# LAB 3: Regresja wieloraka

Prognozowanie cen domów (wiele zmiennych)

W tym notatniku wykorzystamy dane dotyczące sprzedaży domów. Przewidywanie ich ceny zostanie dokonane przy pomocy regresji wielorakiej. Pierwsze zadanie dotyczy eksploracji regresji wielorakiej poprzez opracowanie nowych cech i pomiar błędu. W drugim zadaniu zaimplementować należy algorytm spadku gradientu.

     Użyj wbudowanych funkcji tworzenia wykresów (lub w inny sposób), aby obliczyć wagi regresji (współczynniki)
     Biorąc pod uwagę wagi regresji, predyktory i wynik napisz funkcję obliczającą resztkową sumę kwadratów
     Spójrz na współczynniki i zinterpretuj ich znaczenie
     Oceń wiele modeli za pomocą RSS

# Potrzebne biblioteki

In [1]:
import pandas as pd
import numpy as np

# Wczytanie danych ceny domów 

Zestaw danych pochodzi ze sprzedaży domów w King County, regionie, w którym znajduje się miasto Seattle w stanie Waszyngton.

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
domy = pd.read_csv('kc_house_data.csv',sep=',', dtype=dtype_dict)

In [3]:
domy.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


# Podziel dane na uczące i testowe.
Używamy seed = 0, aby każdy, kto korzysta z tego notebooka, uzyskał te same wyniki. W praktyce możesz ustawić losowy podział.

In [4]:
y = domy['price']
X = domy.drop(['price'], axis=1)
features = X.columns.values
features 

array(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat',
       'long', 'sqft_living15', 'sqft_lot15'], dtype=object)

In [5]:
from sklearn.model_selection import train_test_split

# Dodajemy jedynki na początku
X.insert(0, 'intercept', 1.0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Nauka modelu regresji wielorakiej

Wykorzystując cechy 'sqft_living', 'bedrooms', 'bathrooms' uczymy nasz model.

Po dopasowaniu modelu możemy wyodrębnić współczynniki (wagi) modelu regresji:

In [6]:
example_features = ['intercept', 'sqft_living', 'bedrooms', 'bathrooms']
pinw = np.linalg.pinv(X_train[example_features])
w = np.dot(pinw, y_train)
print('Otrzymane współczynniki:', w)

Otrzymane współczynniki: [ 67512.01513809    313.17055038 -56754.66651422   6887.71910816]


In [7]:
# Upewniamy się, że mamy poprawne współczynniki:

from sklearn.linear_model import LinearRegression

regressor = LinearRegression()  
regressor.fit(X_train[example_features], y_train)

regressor.coef_[0] = regressor.intercept_
print(regressor.coef_)

[ 67512.01513809    313.17055038 -56754.66651422   6887.71910816]


# Przewidywanie wartości modelu

Mając wyliczone parametry modelu napisz funkcję do przewidywania wartości dla zadanego modelu

In [8]:
def predict_output(w, X, v=False):
    a = np.dot(w, X)
    if v:
        print(w.shape, 'x', X.shape, '=', a.shape)
    return a

# Wyliczamy błąd (SSE -  sum of squared estimate of errors)

Teraz, gdy możemy wykonać przewidywania na podstawie modelu, napiszmy funkcję obliczającą RSS modelu. Wykonaj poniższą funkcję, aby obliczy sumę kwadratów błędu estymacji (SSE) na podstawie modelu, danych i wyniku.

In [9]:
def policz_MSE(model, data, y_prawdziwe, features):
    
    # Preparation
    model_p = model.reshape(1, len(model))
    data_p = data[features].transpose()
    y_prawdziwe_p = np.array(y_prawdziwe).reshape(len(y_prawdziwe), 1)
    
    predykcja = predict_output(model_p, data_p, v=True)
    
    roznica = abs(y_prawdziwe_p - predykcja.transpose())

    SSE = np.dot(roznica.transpose(), roznica)
    MSE = SSE / (2 * len(roznica))
    
    print(f'SSE: {SSE[0][0]}\nMSE: {MSE[0][0]}')
    
    return SSE[0][0], MSE[0][0]

Przetestuj swoją funkcję obliczając błąd SSE z danych TEST dla przykładowego modelu:

In [10]:
policz_MSE(w, X_test, y_test, example_features)

(1, 4) x (4, 4323) = (1, 4323)
SSE: 259213572106088.0
MSE: 29980750879.72334


(259213572106088.0, 29980750879.72334)

# Utwórz nowe cechy

Mimo iż nasz model regresji wielorakiej obejmuje wiele różnych cech (np. ilosc_sypiani, powierzchnia i ilosc_lazienek) możemy również rozważyć przekształcenie istniejących cech, np. log(powierzchnia), czy nawet mnożenie ilości sypialni i łazienek.

Użyjemy funkcji logarytmu, aby utworzyć nowe cechy, więc najpierw powinieneś importujemy ją z biblioteki matematycznej.

In [11]:
from math import log

Następnie utwórz następujące 4 nowe cechy jako kolumny w danych TRENINGOWYCH i TESTOWYCH:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
Jako przykład oto pierwsza:

In [12]:
X_train.loc[:,'bedrooms_squared'] = X_train['bedrooms'].apply(lambda x: x**2)
X_test.loc[:,'bedrooms_squared'] = X_test['bedrooms'].apply(lambda x: x**2)

/home/mccastles/snap/code/common/.cache/pypoetry/virtualenvs/workspace-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/mccastles/snap/code/common/.cache/pypoetry/virtualenvs/workspace-py3.7/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
X_train.loc[:,'bed_bath_rooms'] = X_train.apply(lambda row: row['bedrooms'] * row['bathrooms'], axis=1)
X_test.loc[:,'bed_bath_rooms'] = X_test.apply(lambda row: row['bedrooms'] * row['bathrooms'], axis=1)

X_train.loc[:,'log_sqft_living'] = X_train['sqft_living'].apply(lambda x: log(x))
X_test.loc[:,'log_sqft_living'] = X_test['sqft_living'].apply(lambda x: log(x))

X_train.loc[:,'lat_plus_long'] = X_train.apply(lambda row: row['lat'] + row['long'], axis=1)
X_test.loc[:,'lat_plus_long'] = X_test.apply(lambda row: row['lat'] + row['long'], axis=1)

In [14]:
new_features = ['bedrooms_squared', 'bed_bath_rooms', 'log_sqft_living', 'lat_plus_long']
X_test[new_features]

,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
17384,4.0,3.00,7.265430,-74.5678
722,16.0,13.00,8.448914,-74.5290
2680,4.0,1.50,7.272398,-74.6933
18754,4.0,2.00,7.029973,-74.7132
14554,16.0,10.00,8.064636,-74.5053
...,...,...,...,...
5427,16.0,13.00,8.035926,-74.3786
16547,9.0,5.25,7.138867,-74.4346
4585,9.0,7.50,7.215240,-74.4143
17762,9.0,3.00,6.856462,-74.8244


* Kwadrat sypialni zwiększa separację między nielicznymi sypialniami (np. 1) i wieloma sypialniami (np. 4), ponieważ 1 ^ 2 = 1, ale 4 ^ 2 = 16. W konsekwencji ta funkcja będzie miała wpływ głównie na domy z wieloma sypialniami.
* Sypialnia razy łazienka daje tak zwaną funkcję „interakcji”. Wynik jest wysoki, gdy  wartościu *obu* cech są duże.
* Przejęcie logarytmu stóp kwadratowych powoduje zbliżenie dużych wartości i rozłożenie małych wartości. Wynika to z reguły prawoskośności posiadanych danych/posiadanego atrybutu.
* Dodawanie szerokości do długości geograficznej jest całkowicie bezsensowne, ale i tak to zrobimy (zobaczymy później dlaczego)

**Pytanie quizu: Jaka jest średnia (średnia arytmetyczna) twoich 4 nowych funkcji w danych TEST? (w zaokrągleniu do 2 cyfr)**

In [15]:
X_test[new_features].mean()

bedrooms_squared    12.210502
bed_bath_rooms       7.447721
log_sqft_living      7.550239
lat_plus_long      -74.654261
dtype: float64

# Uczenie wielu modeli

Teraz poznamy wagi trzech (zagnieżdżonych) modeli do przewidywania cen domów. Pierwszy model będzie miał najmniej cech, drugi model doda jedną cechę, a trzeci doda jeszcze kilka:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: + bedrooms\*bathrooms
* Model 3: + log squarefeet, bedrooms squared, i (bezsensowne) latitude + longitude

In [16]:
model_1_features = ['intercept', 'sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']
features = [model_1_features, model_2_features, model_3_features]

Teraz, gdy mamy już cechy, poznaj wagi trzech różnych modeli do przewidywania docelowej = „ceny” za pomocą funkcji model_train i spójrz na wartość wag/współczynników:

In [17]:
def model_train(X_train, y_train):
    
    model = np.dot(np.linalg.pinv(X_train), y_train)
    
    return model.reshape(len(model), 1)

In [18]:
model1 = model_train(X_train[model_1_features], y_train)
model2 = model_train(X_train[model_2_features], y_train)
model3 = model_train(X_train[model_3_features], y_train)

In [19]:
models = [model1, model2, model3]
for model in models:
    print(f'Współczynniki dla {len(model)} cech:\n{model}\n')

Współczynniki dla 6 cech:
[[-7.08708462e+07]
 [ 3.12942010e+02]
 [-5.30962691e+04]
 [ 1.47770428e+04]
 [ 6.53983343e+05]
 [-3.25707336e+05]]

Współczynniki dla 7 cech:
[[-6.86068204e+07]
 [ 3.06819573e+02]
 [-1.04604718e+05]
 [-7.01815289e+04]
 [ 6.50590952e+05]
 [-3.09965751e+05]
 [ 2.49441497e+04]]

Współczynniki dla 10 cech:
[[-6.26284503e+07]
 [ 5.37808086e+02]
 [ 2.78047910e+03]
 [ 1.01363766e+05]
 [ 5.30798406e+05]
 [-4.09655435e+05]
 [-1.81822552e+04]
 [ 7.24579939e+02]
 [-5.71030023e+05]
 [ 1.21142971e+05]]



**Quiz: Jaki jest znak (dodatni lub ujemny) dla współczynnika/wagi dla „łazienek” w modelu 1?**

**Quiz: Jaki jest znak (dodatni lub ujemny) dla współczynnika/wagi dla „łazienek” w modelu 2?**

Zastanów się, co to znaczy.

In [20]:
# 1) Dodatni
# 2) Ujemny

# Porównywanie wielu modeli

Teraz, gdy otrzymaliśmy trzy modele i wyodrębniliśmy wagi modeli, chcemy ocenić, który model jest najlepszy.

Używając wcześniej naopisanych funkcji obliczy SSE dla danych uczących dla każdego z trzech modeli.

In [21]:
def SSE_dla_wszystkich(X, y):
    sse_dict = {}
    for i, feature_set, model in zip(range(1, 4), features, models):
        title = f'Model {i}'
        print(f'\n{title}\n({len(feature_set)} features)')
        print(feature_set)
        sse_dict[title] = policz_MSE(model, X, y, feature_set)
    return sse_dict

In [22]:
# Policz SSE na danych TRENINGOWYCH dla każdeg z 3 modeli i zapisz wartości:
sse_train = SSE_dla_wszystkich(X_train, y_train)
sse_train


Model 1
(6 features)
['intercept', 'sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
(1, 6) x (6, 17290) = (1, 17290)
SSE: 979843597588328.6
MSE: 28335558056.342644

Model 2
(7 features)
['intercept', 'sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms']
(1, 7) x (7, 17290) = (1, 17290)
SSE: 970799199729578.2
MSE: 28074008089.345814

Model 3
(10 features)
['intercept', 'sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living', 'lat_plus_long']
(1, 10) x (10, 17290) = (1, 17290)
SSE: 913653644974958.0
MSE: 26421447223.104626


{'Model 1': (979843597588328.6, 28335558056.342644),
 'Model 2': (970799199729578.2, 28074008089.345814),
 'Model 3': (913653644974958.0, 26421447223.104626)}

**Quiz: Który model (1, 2 lub 3) ma najniższy poziom SSE na danych TRENINGOWYCH?** Czy tego się spodziewałeś/-łaś?

In [23]:
min(sse_train, key=sse_train.get)

'Model 3'

Teraz obliczyć SSE na danych TEST dla każdego z trzech modeli.

In [24]:
# Teraz obliczyć SSE na danych TEST dla każdego z trzech modeli i zapisz wartości:
sse_test = SSE_dla_wszystkich(X_test, y_test)
sse_test


Model 1
(6 features)
['intercept', 'sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
(1, 6) x (6, 4323) = (1, 4323)
SSE: 213487129319106.56
MSE: 24692011255.968838

Model 2
(7 features)
['intercept', 'sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms']
(1, 7) x (7, 4323) = (1, 4323)
SSE: 210778544168946.9
MSE: 24378735157.176373

Model 3
(10 features)
['intercept', 'sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long', 'bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living', 'lat_plus_long']
(1, 10) x (10, 4323) = (1, 4323)
SSE: 203972051917624.28
MSE: 23591493397.828392


{'Model 1': (213487129319106.56, 24692011255.968838),
 'Model 2': (210778544168946.9, 24378735157.176373),
 'Model 3': (203972051917624.28, 23591493397.828392)}

**Quiz: Który model (1, 2 lub 3) ma najniższy poziom SSE na danych TESTOWYCH?** Czy tego się spodziewałeś/-łaś? Pomyśl o cechach, które zostały dodane do każdego z modeli.

In [25]:
min(sse_test, key=sse_test.get)

'Model 3'

# Policz pochodną

Przejdziemy teraz do obliczania pochodnej funkcji kosztu regresji. Przypomnij sobie, że funkcja kosztu jest sumą kwadratów różnic między punktami danych a przewidywanym wynikiem.

Ponieważ pochodna sumy jest sumą pochodnych, możemy obliczyć pochodną dla pojedynczego punktu danych, a następnie zsumować na podstawie punktów danych. Możemy zapisać kwadratową różnicę między obserwowanym a przewidywanym wynikiem dla pojedynczego punktu w następujący sposób:

(w[0]\*[CONSTANT] + w[1]\*[cecha_1] + ... + w[i] \*[cecha_i] + ... +  w[k]\*[cecha_k] - output)^2

Gdzie mamy *k* cech i stałą. Tak więc pochodną w odniesieniu do wagi w[i] według reguły łańcucha jest:

2\*(w[0]\*[CONSTANT] + w[1]\*[cecha_1] + ... + w[i] \*[cecha_i] + ... +  w[k]\*[cecha_k] - output)\* [cecha_i]

Pojęcie w nawiasach to tylko błąd (różnica między prognozowaniem a wyjściem). Możemy więc ponownie napisać to jako:

2\*error\*[cecha_i]

Oznacza to, że pochodną wagi cechy *i* jest suma (ponad punkty danych) 2-krotności iloczynu błędu i samej cechy. W przypadku stałej jest to tylko dwukrotność sumy błędów!

Przypomnijmy, że dwukrotność sumy iloczynu dwóch wektorów jest tylko dwukrotnością iloczynu dwóch wektorów. Dlatego pochodna wagi dla *cechy_i* jest tylko dwukrotnością iloczynu między wartościami *cechy_i* a bieżącymi błędami.

Mając to na uwadze, należy wykonać następującą funkcję pochodną, która oblicza pochodną współczynnika na podstawie wartości cechy (we wszystkich punktach danych) i błędów (we wszystkich punktach danych).

In [26]:
# Oblicz podwojony iloczyn cech i błędów, a następnie zwróć otrzymną wartość

def feature_derivative(errors, feature):
    return 2 * np.dot(errors, feature)

# Metoda spadku gradientu / gradientu prostego (Gradient Descent)

Teraz napiszemy funkcję, która wykonuje spadek gradientu. Podstawową przesłanką jest prosta. Biorąc pod uwagę punkt początkowy, aktualizujemy bieżące wagi, przesuwając się w kierunku ujemnego gradientu. Przypomnijmy, że gradient jest kierunkiem *wzrostu*, a zatem gradient ujemny jest kierunkiem *spadku* i staramy się *zminimalizować* funkcję kosztu.

Współczynnik, z jakim poruszamy się w kierunku gradientu ujemnego, nazywa się „rozmiarem kroku” - $\alpha$. Zatrzymujemy się, gdy jesteśmy „wystarczająco blisko” do rozwiązania optymalnego. Definiujemy to, wymagając, aby wielkość (długość) wektora gradientu była mniejsza niż stała „tolerancja” - $\epsilon$.

Mając to na uwadze, wykonaj poniższą funkcję spadku gradientu poniżej, używając powyższej funkcji pochodnej. Dla każdego kroku zejścia gradientu aktualizujemy wagę każdej funkcji przed obliczeniem naszych kryteriów zatrzymania

In [27]:
from math import sqrt

In [28]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    
    
    weights = np.array(initial_weights).reshape(len(initial_weights), 1)
    
    k = 0
    converged = False
    while not converged:
        
        k+=1
        
        # Predykcja
        y_pred = predict_output(feature_matrix, weights)


        # Błąd predykcji
        roznica = y_pred - output

        
        # Aktualizacja wag
        gradient_sum_squares = 0 
        for i in range(len(weights)):
            der = feature_derivative(roznica.transpose(), feature_matrix.iloc[:,i])
            gradient_sum_squares += der ** 2
            weights[i] = weights[i] - step_size * der

            
        # Czy jesteśmy blisko punktu optymalnego
        gradient_magnitude = sqrt(gradient_sum_squares)
        converged = gradient_magnitude < tolerance

        
    print(f'Zminimalizowano funkcję w trakcie {k} kroków.')
    return weights

Kilka rzeczy, na które należy zwrócić uwagę, zanim zaczniemy korzystać z metody spadku gradientu. Ponieważ gradient jest sumą wszystkich punktów danych i obejmuje iloczyn błędu i wartości cechy, sam gradient będzie bardzo duży, ponieważ cechy są duże (stopy kwadratowe), a wynik jest duży (ceny). Tak więc, chociaż można oczekiwać, że „tolerancja” będzie niewielka, mała jest jedynie zależna od wielkości cechy.

Z podobnych powodów rozmiar kroku będzie znacznie mniejszy niż można się spodziewać, ale dzieje się tak, ponieważ gradient ma tak duże wartości.

# Uruchamianie spadku gradientu jako prostej regresji

Chociaż opadanie gradientu jest zaprojektowane dla regresji wielorakiej, ponieważ stała jest teraz funkcją, możemy użyć funkcji spadku gradientu do oszacowania parametrów prostej regresji na "squarefeet". Następująca komórka ustawia funkcję parametr_macierz, wynik, wagi początkowe i rozmiar kroku dla pierwszego modelu:

In [29]:
simple_features = ['intercept', 'sqft_living']
simple_feature_matrix = X_train[simple_features]
# simple_feature_matrix

In [30]:
output = pd.DataFrame(y_train)
# output

In [31]:
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

Następnie uruchom gradient z powyższymi parametrami.

In [32]:
regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

Zminimalizowano funkcję w trakcie 12 kroków.


array([[-46999.88720259],
       [   283.46383063]])

Porównaj wagi do tych uzyskanych przy pomocy pseudoodwrotności?

**Pytanie quizu: Jaka jest waga sqft_living - drugi element „simple_weights” (w zaokrągleniu do 1 miejsca po przecinku)?**

In [33]:
pinw = np.linalg.pinv(simple_feature_matrix)
w = np.dot(pinw, y_train)
print('Otrzymane współczynniki:', w)
print('Waga sqft_living = ', round(w[1], 1))

Otrzymane współczynniki: [-48257.06359103    283.96855716]
Waga sqft_living =  284.0


Zobacz jak się zachowuje metoda gradientowa po przeskalowaniu wartości cech:

In [34]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(pd.DataFrame(X_train['sqft_living']))
scaled = pd.DataFrame(scaled, columns=['sqft_living'])
scaled.insert(0, 'intercept', 1.0)
scaled

,intercept,sqft_living
0,1.0,0.096604
1,1.0,0.112453
2,1.0,0.060377
3,1.0,0.144906
4,1.0,0.113962
...,...,...
17285,1.0,0.276981
17286,1.0,0.083774
17287,1.0,0.156226
17288,1.0,0.156981


In [35]:
# Trwa za długo
# regression_gradient_descent(scaled, output, initial_weights, step_size, tolerance)

In [37]:
# Zwiększamy krok
step_size = 7e-7
regression_gradient_descent(scaled, output, initial_weights, step_size, tolerance)

Zminimalizowano funkcję w trakcie 27723 kroków.


array([[  54505.09951064],
       [3612333.49094962]])